# MLighter

Copyright 2023 Aidan Dakhama, Hector D. Menendez  

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Importing frontend library and initialising session

In [47]:
import threading

import numpy as np
import bqplot.pyplot as plt
import pandas as pd

from frontend_lib import *

session = initialise_session()

## Base class for pages

In [48]:
class Page(Testable):
    """
    Base class for pages.
    When adding a new page inherit from this class and implement the methods below which raise NotImplementedError.
    the help method is optional, if you do not wish to provide help for the page, do not implement it.
    
    When you have finished with a page, ensure you call the add_navigation_pages method to add the page to the navigation bar. i.e. any pages you want to be reachable from this page should be added here, in order.
    """

    def __init__(self):
        super().__init__()
        self.navigation_pages = []

    def content(self):
        """Return the content to be displayed on the page"""
        raise NotImplementedError()

    def title(self):
        """Return the title to be displayed in the header"""
        raise NotImplementedError()

    def subtitle(self):
        """Return the subtitle to be displayed in the header"""
        raise NotImplementedError()

    def navigation(self):
        """Return a list of pages to be displayed in the navigation bar"""
        return self.navigation_pages

    def add_navigation_pages(self, *pages):
        """Add pages to the navigation bar, add them in the order you want them to appear, including this page"""
        self.navigation_pages.extend(pages)

    def name(self):
        """Unique name used in navigation and by the site class to index pages"""
        raise NotImplementedError()

    def icon(self):
        """Icon to be displayed in the navigation bar"""
        raise NotImplementedError()

    def help(self):
        """Return either None, or if you wish to provide help for the page return some help content."""
        return None

    def __repr__(self):
        """To help with debugging"""
        return f"Page \n (\n\tname: {self.name()}\n\ttitle: {self.title()}\n\tsubtitle: {self.subtitle()}\n\ticon: {self.icon()}\n)"


## Site Class

In [49]:
class Site(Testable):
    """
    A class to manage the state of the site.
    You register a single home page, and add other pages to the site.
    You then call the content method to get the content to be displayed.
    This class manages all navigation.
    """

    def __init__(self):
        super().__init__()
        self.content_area = v.Layout(
            _metadata={'mount_id': 'content-main'},
            children=["loading..."]
        )

        self.content_title = v.ToolbarTitle(
            _metadata={'mount_id': 'content-title'},
            children=['MLighter']
        )

        self.content_subtitle = v.Layout(
            _metadata={'mount_id': 'content-bar'},
            children=['MLighter Main Menu']
        )

        self.navigation_bar = v.List(
            _metadata={'mount_id': 'content-nav'},
            column=True,
            children=["loading..."]
        )

        self.home = None
        self.current_page = self.home

    def set_navigation(self, pages):
        """
        Set the navigation bar to contain the given pages, and sets up the event handler
        """
        link_components = []

        for page in pages:
            component = v.ListItem(
                link=True,
                children=[
                    v.ListItemAction(children=[
                        v.Icon(children=[page.icon()]),
                    ]),
                    v.ListItemContent(children=[
                        v.ListItemTitle(children=[page.name()]),
                    ]),
                ],
            )

            on_click = lambda *ignored, page_to_switch=page: self.switch_page(page_to_switch)

            component.on_event('click', on_click)

            link_components.append(component)

        self.navigation_bar.children = link_components

    def set_page_header(self):
        """
        Sets the header to the current pages, including any help if the page provides it
        """
        help_content = self.current_page.help()

        if help_content is None:
            self.content_subtitle.children = [
                v.ToolbarTitle(children=[self.current_page.subtitle()]),
            ]
        else:
            dialog = help_dialog(self.current_page.name(), help_content)
            self.content_subtitle.children = [
                v.ToolbarTitle(children=[self.current_page.subtitle()]),
                v.Spacer(),
                dialog,
            ]

    def set_home(self, home_page):
        """Sets the home page of the site, this will be the page we render initially"""
        self.home = home_page
        self.current_page = self.home

    def set_page(self):
        """Renders the current page and all related components"""
        if self.current_page is None:
            if self.home is not None:
                self.current_page = self.home
            else:
                raise Exception("No home page set")

        self.content_area.children = [self.current_page.content()]
        self.content_title.children = [self.current_page.title()]
        self.set_navigation(self.current_page.navigation())
        self.set_page_header()

    def switch_page(self, page):
        """Updates the current page"""
        self.current_page = page
        self.set_page()

    def content(self):
        """Returns the content to be displayed"""
        self.set_page()
        return self.content_area

## Initialising the site

In [50]:
site = Site()

## Page classes

classes that both contain the page content and construct them as vuetify components.

### Home page

Create the home page, which will be displayed when the user first opens the application.

In [51]:
class Home(Page):  
    introduction = "Welcome to MLighter, your holistic tool to test machine learning implementations. Here, you have two main options, testing your code or testing you machine learning model."

    code_section_1 = "If you want to "
    code_bold = "test your code"
    code_section_2 = ", you can either run an active testing or perform a code review using our database of vulnerabilities in machine learning implementations."

    model_section_1 = "If you want to "
    model_bold = "test your model"
    model_section_2 = ", you just need to upload your model with some data and select and adversarial scenario. MLighter will aim to identify variants to your model based on your criteria to measure its robustness."

    image = "./logo.png"
    image_alt = "MLighter logo"
    image_width = 200

    def title(self):
        return "MLighter"

    def subtitle(self):
        return "Mlighter Main Menu"

    def name(self):
        return "Home"

    def icon(self):
        return "home"

    def content(self):
        return v.Layout(
            row=True,
            wrap=True,
            align_center=True,
            justify_center=True,
            class_='mx-2 my-2',
            children=[
                v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                    centre(image(self.image, self.image_alt, self.image_width)),
                    vspace(),
                    centre(heading(self.title(), 1)),
                    vspace(),
                    paragraph(self.introduction),
                    vspace(),
                    paragraph(self.code_section_1, bold(self.code_bold), self.code_section_2),
                    vspace(),
                    paragraph(self.model_section_1, bold(self.model_bold), self.model_section_2),
                ]),
            ],
        )

In [52]:
home = Home()

### Add Home page to site

In [53]:
site.set_home(home)

## Code Testing pages

In [54]:
class CodeTestingArea:
    """
    State to be shared between the pages of the code testing section
    """
    bug_db = {}

    bug_db_select = v.Select(
        v_model="None",
        label="Select a Bug from the Database",
        items=["None"],
    )

    bug_run_select = v.Select(
        v_model="None",
        label="Select a Bug from the Tester",
        items=["None"],
    )

### Code Testing

In [55]:

class CodeTesting(Page):
    help_content = "To test your code, you need a template calling it. Please see our available templates to see the proper format. Once uploaded you can run your code and see the results in the graphs.\nAdditionally, you can upload any auxiliary files you may need which will be placed in same directory as your template."

    def help(self):
        return v.Layout(
            row=True,
            wrap=True,
            align_center=True,
            justify_center=True,
            children=[
                v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                    paragraph(self.help_content),
                ]),
            ],
        )

    def title(self):
        return "Code Blind Spots"

    def subtitle(self):
        return "Code Testing"

    def name(self):
        return "Code Testing"

    def icon(self):
        return "account_box"

    def __init__(self):
        super().__init__()
        self.bugs_run = {}

        self.input_uploader = widgets.FileUpload(
            accept='',
            multiple=False,
            tooltip='Upload your input file',
        )
        self.template_uploader = widgets.FileUpload(
            accept='',
            multiple=False,
            tooltip='Upload your template file',
        )
        self.template_aux_uploader = widgets.FileUpload(
            accept='',
            multiple=True,
            tooltip='Upload any code files your template needs',
        )
        self.test_selector = widgets.Dropdown(
            options=[x[0] + (" (running)" if x[1] else " (not running)") for x in
                     session.getAllTests(displayRunning=True)],
            value=None,
            disabled=False,
        )
        self.name_input = widgets.Text(
            value='',
            placeholder='Type a test name',
            disabled=False,
        )

        # Plots
        n = 100
        self.x = np.linspace(0.0, 10.0, n)
        self.y = np.array([0])

        self.fig_crashes, self.hist_crashes = self.create_plot("Crashes")
        self.fig_hangs, self.hist_hangs = self.create_plot("Hangs")
        self.fig_executions, self.hist_executions = self.create_plot("Executions")
        self.fig_paths, self.hist_paths = self.create_plot("Paths")

        # Buttons
        self.code_test_button = v.Btn(children=['Start Testing'])
        self.refresh_tests_button = v.Btn(children=['Refresh Tests'])
        self.run_crashes_button = v.Btn(children=['Run Crashes'])
        self.new_test_button = v.Btn(children=['New Test'])

        # status
        self.status = paragraph("Not Running")

        self.register_callbacks()

    def create_plot(self, name):
        fig = plt.figure(title=name)
        np.random.seed(0)
        hist = plt.plot(self.x, self.y)

        fig.layout.width = 'auto'
        fig.layout.height = 'auto'
        fig.layout.min_width = '300px'
        fig.layout.min_height = '300px'

        return fig, hist

    def register_callbacks(self):
        # TODO: Make more robust, i.e. check if both an input and code was provided before enabling the code test button
        def refresh_test_selections(*ignored):
            self.test_selector.options = [x[0] + (" (running)" if x[1] else " (not running)") for x in session.getAllTests(displayRunning=True)]

        def view_previous_test(change):
            if change['type'] == 'change' and change['name'] == 'value':
                if change['new'] is not None:
                    session.reconnectToSession(self.test_selector.value.split(" ")[0])

                    y = np.array([0])
                    self.hist_crashes.y = self.hist_hangs.y = self.hist_executions.y = self.hist_paths.y = y

                    self.status.children = "viewing test " + self.test_selector.value.split(" ")[0]
                    self.update_hists(session)

        def set_test_name():
            if not self.name_input.disabled:
                if self.name_input.value == "":
                    self.name_input.value = session.setTestName()
                else:
                    self.name_input.value = session.setTestName(self.name_input.value)
                self.name_input.disabled = True

        def upload_input(*ignored):
            set_test_name()
            uploaded_file = self.input_uploader.value[0]['content']
            code_string = uploaded_file
            session.uploadCodeInput(codeContent=code_string)

        def upload_template(*ignored):
            set_test_name()
            uploaded_file = self.template_uploader.value[0]['content']
            uploaded_name = self.template_uploader.value[0]['name']
            code_string = str(uploaded_file.tobytes(), 'ascii')
            if uploaded_name.lower().endswith(".py"):
                session.uploadCodeTemplate("python", codeContent=code_string)
            elif uploaded_name.lower().endswith(".r"):
                session.uploadCodeTemplate("R", codeContent=code_string)
            else:
                #TODO: Error message
                return
            self.code_test_button.disabled = False
            
        def upload_aux_files(*ignored):
            set_test_name()
            aux_files = []
            
            for file in self.template_aux_uploader.value:
                aux_files.append({
                    "name": file['name'],
                    "content":str(file['content'].tobytes(), 'ascii')
                })
            
            session.upload_aux_files(aux_files)

        def run_crashes(*ignored):
            self.status.children = "Crashes Generated"
            command = "bash runner.sh " + session.codeTemplate + " " + session.outputsFolder
            os.system(command)
            for i in range(0, session.crashes):
                new_case = {}

                f_out = open(f"crash_{i}.txt", "r")
                f_err = open(f"crash_error_{i}.txt", "r")

                lines_out = f_out.readlines()
                lines_err = f_err.readlines()

                new_case["Output"] = lines_out
                new_case["Error"] = lines_err
                self.bugs_run[f"Crash{i}"] = new_case

            CodeTestingArea.bug_run_select.items = list(self.bugs_run.keys())

        def test_code(*ignored):
            self.code_test_button.disabled = True
            self.new_test_button.disabled = False
            session.runCodeTesting()
            self.status.children  = "Code Running"
            self.update_hists(session)

        def new_test(*ignored):
            session.resetSession()
            self.name_input.disabled = False
            self.code_test_button.disabled = False
            self.status.children = "Not Running"
            self.name_input.value = ""

            self.y = np.array([0])
            self.hist_crashes.y = self.hist_hangs.y = self.hist_executions.y = self.hist_paths.y = self.y

            self.update_hists(session)

        # Register callbacks
        self.refresh_tests_button.on_event('click', refresh_test_selections)
        self.test_selector.observe(view_previous_test)
        self.input_uploader.observe(upload_input, 'value')
        self.template_uploader.observe(upload_template, 'value')
        self.template_aux_uploader.observe(upload_aux_files, 'value')
        self.run_crashes_button.on_event('click', run_crashes)
        self.code_test_button.on_event('click', test_code)
        self.new_test_button.on_event('click', new_test)

    def update_hists(self, current_session):
        try:
            state = current_session.retrieveTestingState()
            if (not state is None) and (len(self.hist_crashes.y) < 100):
                    self.hist_crashes.y = np.concatenate((self.hist_crashes.y, np.array([int(state["Crashes"])])))
                    current_session.crashes=int(state["Crashes"])
                    self.hist_hangs.y = np.concatenate((self.hist_hangs.y, np.array([int(state["Hangs"])])))
                    self.hist_executions.y = np.concatenate((self.hist_executions.y, np.array([int(state["Execs"])])))
                    self.hist_paths.y = np.concatenate((self.hist_paths.y, np.array([int(state["Paths"])])))  
                    
                    if int(state["Crashes"]) > 0:
                        self.run_crashes_button.disabled = False
        except FileNotFoundError:
            print("Waiting for the tester to be ready\n")

        threading.Timer(1, self.update_hists, (current_session,)).start()

    def content(self):
        return tabs(tab_headers=["Select or Start Test", "Upload and Run", "Monitor"], tab_items=[
            v.Row(cols=3, md=1, class_='justify-center mx-2 my-2', children=[
                div(centre(heading("Test name", 3)), 
                    div(self.name_input, class_='mx-8 my-8'), 
                    class_='mx-2 my-2'
                ),
                div(centre(heading("Reconnect", 3)), 
                    div(self.test_selector, class_='mx-8 my-8'), 
                    class_='mx-2 my-2'
                ),
                div(centre(heading("Start new test", 3)),
                    div(centre(self.new_test_button), class_='mx-8 my-8'), 
                    class_='mx-2 my-2'
                    ),
            ]),
            v.Row(cols=3, md=1, class_='justify-center mx-2 my-2', children=[
                div(centre(heading("Upload", 3)), div(
                    v.Row(children=[
                        v.Col(children=[
                            centre(paragraph("Input file")),
                            self.input_uploader,
                        ]),
                        v.Col(children=[
                            centre(paragraph("Template file")),
                            self.template_uploader,
                        ]),
                        v.Col(children=[
                            centre(paragraph("Auxiliary files")),
                            self.template_aux_uploader,
                        ]),
                    ])),
                    class_='mx-8 my-8'
                    ),
                div(centre(heading("Run test", 3)),
                    div(centre(self.code_test_button), class_='mx-8 my-8'),
                    div(centre(heading("Code status", 3)), div(self.status, class_='mx-8 my-8'), class_='mx-8 my-8'),
                    class_='mx-2 my-2'
                ),
            ]),
            v.Layout(
                row=True,
                wrap=True,
                align_center=True,
                justify_center=True,
                children=[
                    v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                        v.Row(cols=2, lg=2, md=1, class_='justify-center', children=[
                            div(self.fig_crashes, class_='mx-2 my-2'),
                            div(self.fig_hangs, class_='mx-2 my-2'),
                        ]),
                        v.Row(cols=2, lg=2, md=1, class_='justify-center', children=[
                            div(self.fig_executions, class_='mx-2 my-2'),
                            div(self.fig_paths, class_='mx-2 my-2'),
                        ]),
                    ]),
                ]),
        ])

In [56]:
code_testing = CodeTesting()
code_testing.content()

### Code Review

In [57]:
class CodeReview(Page):
    help_content = "If you want to know whether your code is sensitive to potential vulnerabilities, you can upload your code here and we will check it against our database of ML bugs, vulnerabilities, and exploits and it will allow you to identify blind spots in your code."

    def help(self):
        return v.Layout(
            row=True,
            wrap=True,
            align_center=True,
            justify_center=True,
            children=[
                v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                    paragraph(self.help_content),
                ]),
            ],
        )

    def title(self):
        return "Code Blind Spots"

    def subtitle(self):
        return "Code Review"

    def name(self):
        return "Code Review"

    def icon(self):
        return "timeline"

    def __init__(self):
        super().__init__()
        self.temporal_db = pd.read_csv("tablaPython.csv")  # TODO: Add R

        self.code_uploader = widgets.FileUpload(
            accept='',
            multiple=False
        )

        # Table
        self.table_imports = v.DataTable(
            headers=[],
            v_model=[],
            item_key="function",
            single_select=False,
            show_select=True,
            items=[]
        )

        # Text
        self.calls = paragraph("")
        self.imports = paragraph("")

        # Buttons
        self.code_review_button = v.Btn(children=['Review code'])
        self.code_review_button.disabled = True

        self.load_reports_button = v.Btn(children=['Load reports'])
        self.load_reports_button.disabled = True

        self.register_callbacks(session)

    def register_callbacks(self, current_session):
        def upload_code(*ignored):
            uploaded_file = self.code_uploader.value[0]["content"]
            code_string = uploaded_file
            uploaded_file_name = self.code_uploader.value[0]["name"]

            if uploaded_file_name.lower().endsWith(".py"):
                current_session.uploadCodeReview("python", codeContent=code_string)
            elif uploaded_file_name.lower().endsWith(".r"):
                current_session.uploadCodeReview("R", codeContent=code_string)

            self.code_review_button.disabled = False
            self.load_reports_button.disabled = False

        def review_code(*ignored):
            import_funcs = current_session.evaluateCodeReview()
            self.imports.children = str(import_funcs)

            import_data = pd.DataFrame(import_funcs)
            known_bugs = unknown_bugs = []

            for import_element in import_data["function"]:
                if isinstance(import_element, list):
                    import_element = import_element[0]

                known_bugs = known_bugs + [self.temporal_db[
                                               (self.temporal_db["Function"] == import_element) &
                                               (self.temporal_db["Known"] == "Y")
                                               ].count()[0]]
                unknown_bugs = unknown_bugs + [self.temporal_db[
                                                   (self.temporal_db["Function"] == import_element) &
                                                   (self.temporal_db["Known"] == "N")
                                                   ].count()[0]]

                import_data["Known bugs"] = known_bugs
                import_data["Unknown bugs"] = unknown_bugs

                headers = list(map(lambda x: {'text': x, 'value': x}, import_data.columns))
                self.table_imports.headers = headers
                self.table_imports.items = import_data.to_dict('records')

        def load_reports(*ignored):
            counter = {}
            for record in self.temporal_db.to_dict("records"):
                if record["Function"] not in counter.keys():
                    counter[record["Function"]] = 1
                else:
                    counter[record["Function"]] = counter[record["Function"]] + 1
                CodeTestingArea.bug_db[record["Function"] + str(counter[record["Function"]])] = record

            CodeTestingArea.bug_db_select.items = list(CodeTestingArea.bug_db.keys())

        self.code_uploader.observe(upload_code, 'value')
        self.code_review_button.on_event('click', review_code)
        self.load_reports_button.on_event('click', load_reports)

    def content(self):
        return v.Layout(
            row=True,
            wrap=True,
            align_center=True,
            justify_center=True,
            class_='mx-2 my-2',
            children=[
                v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                    v.Row(cols=1, lg=1, md=1, class_="my-8 justify-center", children=[
                        self.code_uploader
                    ]),
                    v.Row(cols=2, lg=2, md=1, class_="my-8", children=[
                        self.code_review_button,
                        v.Spacer(),
                        self.load_reports_button,
                    ]),
                    v.Row(cols=1, lg=1, md=1, class_="my-8 justify-center", children=[
                        self.table_imports
                    ])
                ]),
            ])



In [58]:
code_review = CodeReview()

### Bug Reports

In [59]:
class BugReports(Page):
    help_content = "This part shows the results in terms of bugs and exploit-ability. Select a bug report then navigate to 'Bug Report' to view it."

    def help(self):
        return v.Layout(
            row=True,
            wrap=True,
            align_center=True,
            justify_center=True,
            children=[
                v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                    paragraph(self.help_content),
                ]),
            ],
        )

    def title(self):
        return "Code Blind Spots"

    def subtitle(self):
        return "Bug Reports"

    def name(self):
        return "Bug Reports"

    def icon(self):
        return "bug_report"

    def __init__(self):
        super().__init__()

        self.bug_report = v.Container(children=[])

    def register_callbacks(self):
        def update_bug(change):
            self.set_bug_report(CodeTestingArea.bug_db[change["new"]])

        def update_output(change):
            self.set_bug_output(CodeTestingArea.bug_db[change["new"]])

        CodeTestingArea.bug_db_select.observe(update_bug, 'v_model')
        CodeTestingArea.bug_run_select.observe(update_output, 'v_model')

    def content(self):
        return tabs(tab_headers=['Load Bugs', 'Bug Report'], tab_items=[
            v.Layout(
                row=True,
                wrap=True,
                align_center=True,
                justify_center=True,
                class_='mx-2 my-2',
                children=[
                    v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                        v.Row(cols=1, lg=1, md=1, class_="my-8 justify-center", children=[
                            CodeTestingArea.bug_db_select,
                            v.Spacer(),
                            CodeTestingArea.bug_run_select
                        ]),
                    ]),
                ]),
            v.Layout(
                row=True,
                wrap=True,
                align_center=True,
                justify_center=True,
                class_='mx-2 my-2',
                children=[
                    v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                        v.Row(cols=1, lg=1, md=1, class_="my-8 justify-center", children=[
                            self.bug_report,
                        ]),
                    ]),
                ]),
        ])

    def set_bug_report(self, bug_case):
        title = heading("Bug Report", 1)
        language = paragraph(f"Language: {str(bug_case['Language (R/Python)'])}")
        version = paragraph(f"Version: {str(bug_case['Language Version'])}")
        known = paragraph(f"Known: {str(bug_case['Known'])}")
        library = paragraph(f"Library: {str(bug_case['Library'])}")
        function = paragraph(f"Function: {str(bug_case['Function'])}")
        if bug_case["Method"] is not float('nan'):
            method = paragraph(f"Method: {str(bug_case['Method'])}")
        else:
            method = paragraph(f"Method: Not an Object")
        line = paragraph(f"Line: {str(bug_case['Line'])}")
        bug_file = paragraph(f"File: {str(bug_case['File'])}")
        bug_input = paragraph(f"Input: {str(bug_case['Input'])}")
        error_type = paragraph(f"Error Type: {str(bug_case['Error Type'])}")
        message = paragraph(f"Output: {str(bug_case['Message'])}")
        exploit = paragraph(f"Exploit: {str(bug_case['Exploit Type'])}")

        self.bug_report.children = [title, language, version, known, library, function, method, line, bug_file,
                                    bug_input, error_type, message, exploit]

    def set_bug_output(self, run_case):
        title = heading("Output Report", 1)
        output = heading("Output Produced: ", 3)
        output_cases = []
        for line in run_case["Output"]:
            output_cases.append(paragraph(line))

        error_message = heading("Error Produced: ", 3)
        error_cases = []
        for line in run_case["Error"]:
            error_cases.append(paragraph(line))

        self.bug_report.children = [title, vspace(), output] + output_cases + [error_message] + error_cases


In [60]:
bug_reports = BugReports()

### Add Code Testing pages to site

In [61]:
code_testing_links = [
    home,
    code_testing,
    code_review,
    bug_reports
]

code_testing.add_navigation_pages(*code_testing_links)
code_review.add_navigation_pages(*code_testing_links)
bug_reports.add_navigation_pages(*code_testing_links)

# Add link to first page of area to homes links
home.add_navigation_pages(code_testing)

## Model Testing Pages

In [62]:
class ModelTestingArea:
    evasion_cases = {}

    evaluation_report_select = v.Select(
        v_model="None",
        label="Select Report From Session",
        items=["None"],
    )


### Dataset

In [77]:
class Dataset(Page):
    help_content_paragraph_1 = "First you need to load valid data for your model, MLighter will use this data to create variants according to the different adversarial scenarios."

    help_content_paragraph_2 = "Once you have uploaded the data, check that the features chosen are the ones that you want to use. If there are any features that you need to remove, please go to 'Clean and Select Features' and remove any unnecessary features."

    help_content_paragraph_3 = "You also need to select the class or target feature, please also do this in 'Clean and Select Features'."

    def help(self):
        return v.Layout(
            row=True,
            wrap=True,
            align_center=True,
            justify_center=True,
            children=[
                v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                    paragraph(self.help_content_paragraph_1),
                    paragraph(self.help_content_paragraph_2),
                    paragraph(self.help_content_paragraph_3),
                ]),
            ],
        )

    def title(self):
        return "Model Blind Spots"

    def subtitle(self):
        return "Dataset"

    def name(self):
        return "Dataset"

    def icon(self):
        return "account_box"

    def __init__(self):
        super().__init__()

        self.dataset_uploader = widgets.FileUpload(
            accept='',
            multiple=False
        )

        self.table_features = v.DataTable(
            headers=[],
            class_="elevation-1",
            v_model=[],
            item_key="Feature",
            single_select=False,
            show_select=True,
            items=[]
        )

        self.remove_feature_button = v.Btn(children=["Remove Features"])
        self.target_feature_button = v.Btn(children=["Target Feature"])
        self.remove_feature_button.disabled = True
        self.target_feature_button.disabled = True

        self.table_data = v.DataTable(
            headers=[],
            class_="elevation-1",
            items=[]
        )

        self.dataset_loaded_text = paragraph("No dataset loaded")
        
        self.stop_upload_button = v.Btn(children=["Stop Upload"], style_="mx-4")
        self.should_upload_button = v.Btn(children=["Upload anyway but don't display"], style_="mx-4")
        self.should_display_button = v.Btn(children=["Upload and display"], style_="mx-4")
        
        self.dialog = warning_dialog(
            div(
                paragraph("The file you are trying to upload is larger than 100MB."),
                paragraph("Do you want to continue?"),
                v.Row(children=[
                    v.Col(class_="my-2 justify-center", children=[centre(self.stop_upload_button)]), 
                    v.Col(class_="my-2 justify-center", children=[centre(self.should_upload_button)]), 
                    v.Col(class_="my-2 justify-center", children=[centre(self.should_display_button)]), 
                ])
            ),
        )

        self.register_callbacks()

    def register_callbacks(self):
        def update_dataset(*ignored):
            if self.dataset_uploader.value == ():
                # We need to be careful here as in should_not_upload we clear the upload button, and must ensure
                # we don't try to handle the empty case, but the api for this and clearing an uploader are not very stable
                # so may change.
                return 
                        
            uploaded_file = self.dataset_uploader.value[0]['content']
            uploaded_file_name = self.dataset_uploader.value[0]['name']
            
            def close_dialog():
                self.dialog.value = True
                self.dialog.value = False
            
            def should_not_upload(*ignored):
                close_dialog()
                self.dataset_uploader.value = []
                
            def should_not_display(*ignored):
                close_dialog()
                session.uploadDataset("structured",
                                  datasetName=uploaded_file_name,
                                  dataFile=uploaded_file
                                  )
                
                self.target_feature_button.disabled = False
                self.remove_feature_button.disabled = False
    
                self.dataset_loaded_text.children = f"Data ready\n{repr(session.data.data)}"

                
            def should_upload(*ignored):
                close_dialog()
                session.uploadDataset("structured",
                                  datasetName=uploaded_file_name,
                                  dataFile=uploaded_file
                                  )
                
                self.update_tables()
                
                self.target_feature_button.disabled = False
                self.remove_feature_button.disabled = False
    
                self.dataset_loaded_text.children = f"Data ready\n{repr(session.data.data)}"

            self.stop_upload_button.on_event('click', should_not_upload)
            self.should_upload_button.on_event('click', should_not_display)
            self.should_display_button.on_event('click', should_upload)
            
            # Check the size of the uploaded file 
            if self.dataset_uploader.value[0]['size'] > 100_000_000:
                self.dialog.value = True
            else:
                should_upload()

        def remove_features(*ignored):
            for element in self.table_features.v_model:
                session.data.cleanColumn(element["Feature"])
            self.table_features.v_model = []

            self.update_tables()

        def target_features(*ignored):
            for element in self.table_features.v_model:
                session.data.setTarget(element["Feature"])
            self.table_features.v_model = []

            self.target_feature_button.disabled = True

        self.dataset_uploader.observe(update_dataset, 'value')
        self.remove_feature_button.on_event('click', remove_features)
        self.target_feature_button.on_event('click', target_features)

    def update_tables(self):
        headers = list(map(lambda x: {'text': x, 'value': x}, session.data.getColumns()))
        items = session.data.data.to_dict('records')
        self.table_data.headers = headers
        self.table_data.items = items

        items = list(map(lambda x: {'Feature': x}, session.data.getColumns()))
        self.table_features.headers = [{'text': 'Feature', 'value': 'Feature'}]
        self.table_features.items = items

    def content(self):
        return tabs(tab_headers=["Upload Data", "Clean and Select Features"],
                    tab_items=[
                        v.Layout(
                            row=True,
                            wrap=True,
                            align_center=True,
                            justify_center=True,
                            class_='mx-2 my-2',
                            children=[
                                v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                                    v.Col(class_="my-8 justify-center", children=[
                                        centre(paragraph(
                                            "Please provide at least one sample from your dataset, including headers.")),
                                        vspace(),
                                        centre(div(self.dataset_uploader)),
                                        vspace(),
                                        self.table_data,
                                        self.dialog
                                    ]),
                                ]),
                            ]
                        ),
                        v.Layout(
                            row=True,
                            wrap=True,
                            align_center=True,
                            justify_center=True,
                            class_='mx-2 my-2',
                            children=[
                                v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                                    v.Row(cols=2, lg=2, md=1, class_="my-8 justify-center", children=[
                                        v.Col(class_="my-8 justify-center", children=[
                                            centre(
                                                paragraph("Remove those classes that are not relevant for the model")),
                                            centre(self.remove_feature_button),
                                        ]),
                                        v.Col(class_="my-8 justify-center", children=[
                                            centre(paragraph("Please select the feature that represents the class")),
                                            centre(self.target_feature_button),
                                        ]),
                                        self.table_features,
                                    ]),
                                ]),
                            ]
                        ),
                    ]
                    )

In [78]:
dataset = Dataset()

Tabs(children=[Tab(children=['Upload Data'], layout=None), Tab(children=['Clean and Select Features'], layout=…

### Model

In [21]:
class Model(Page):
    help_content = "In this section, you can upload your model. In order to check whether the data and model are compatible, you can use the prediction and evaluate whether there is a valid prediction."

    def help(self):
        return v.Layout(
            row=True,
            wrap=True,
            align_center=True,
            justify_center=True,
            children=[
                v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                    paragraph(self.help_content),
                ]),
            ],
        )

    def title(self):
        return "Model Blind Spots"

    def subtitle(self):
        return "Model"

    def name(self):
        return "Model"

    def icon(self):
        return "timeline"

    def __init__(self):
        super().__init__()

        self.model_uploader = widgets.FileUpload(
            accept='',
            multiple=False
        )

        self.pre_evaluation_button = v.Btn(children=["Pre-evaluation"])
        self.pre_evaluation_button.disabled = True

        self.pre_evaluation_table = v.DataTable(
            headers=[
                {'text': 'Expected', 'value': 'Expected'},
                {'text': 'Predicted', 'value': 'Predicted'}],
            class_="elevation-1",
            items=[]
        )

        self.model_loaded_text = paragraph("No model loaded")

        self.register_callbacks()

    def register_callbacks(self):
        def upload_model(*ignored):
            uploaded_file = self.model_uploader.value[0]['content']
            uploaded_file_name = self.model_uploader.value[0]['name']

            session.uploadModel("sklearn", uploaded_file_name, modelFile=uploaded_file)

            self.pre_evaluation_button.disabled = False

            self.model_loaded_text.children = "Model ready"

        def pre_evaluation(*ignored):
            predictions = session.prediction(session.data.data)

            prediction_data_frame = pd.DataFrame()
            prediction_data_frame["Expected"] = list(session.data.target)
            prediction_data_frame["Predicted"] = predictions

            coincidences = prediction_data_frame["Expected"] == prediction_data_frame["Predicted"]
            session.data.originalAccuracy = sum(coincidences)

            self.pre_evaluation_table.items = prediction_data_frame.to_dict('records')

        self.model_uploader.observe(upload_model, 'value')
        self.pre_evaluation_button.on_event('click', pre_evaluation)

    def content(self):
        return v.Layout(
            row=True,
            wrap=True,
            align_center=True,
            justify_center=True,
            class_='mx-2 my-2',
            children=[
                v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                    v.Row(cols=2, lg=2, md=1, class_="my-8; justify-center", children=[
                        v.Col(class_="my-8 justify-center", children=[
                            centre(paragraph("Please provide the model you want to evaluate")),
                            centre(self.model_uploader),
                        ]),
                        v.Col(class_="my-8 justify-center", children=[
                            centre(paragraph("Please select the feature that represents the class")),
                            centre(self.pre_evaluation_button),
                        ]),
                        self.pre_evaluation_table,
                    ]),
                ]),
            ]
        )

In [22]:
model = Model()

### Evasion

In [23]:
class Evasion(Page):
    help_content = "In this part, you have different evasion scenarios where you can choose and configure the most likely to affect your model. Some scenarios generate random variants while others use a learning strategy. Once you have it ready, go to run the run tab to create variants. If you want to see the effect of these variants in your model, go to reports."

    def help(self):
        return v.Layout(
            row=True,
            wrap=True,
            align_center=True,
            justify_center=True,
            children=[
                v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                    paragraph(self.help_content),
                ]),
            ],
        )

    def title(self):
        return "Model Blind Spots"

    def subtitle(self):
        return "Evasion"

    def name(self):
        return "Evasion"

    def icon(self):
        return "build" 

    def __init__(self):
        super().__init__()

        self.configuration = {
            "numberVariants": 1
        }

        self.evasion_strategy_select = v.Select(
            v_model="Random Noise",
            label="Evasion Strategy",
            items=[
                "Random Noise",
                "General Search",
                "Target Class Search",
            ]
        )

        self.evasion_select = v.Select(
            v_model="None",
            label="Type of Evasion",
            items=[
                "None",
                "Discrete",
                "Continuous",
            ]
        )

        self.number_variants_slider = slider(1, 10, True)

        self.discrete_noise_slider = slider(0, 10, True)
        self.continuous_noise_slider = slider(0, 10, False)

        self.discrete_shift_slider = slider(0, 10, True)
        self.continuous_shift_slider = slider(0, 10, False)

        self.select_features_button = v.Btn(children=["Select Features"])

        self.evasion_table = v.DataTable(
            headers=[],
            class_="elevation-1",
            v_model="selected",
            item_key="Feature",
            single_select=False,
            show_select=True,
            items=[]
        )

        self.evasion_container = v.Container(children=[
            self.evasion_strategy_select,
            self.evasion_select,
            paragraph("Select the number of variants per input:"),
            self.number_variants_slider,
        ])

        # RUN section
        self.run_button = v.Btn(children=["Run"])

        self.variants_table = v.DataTable(
            headers=[],
            clsas_="elevation-1",
            items=[]
        )

        self.register_callbacks()

        session.chooseStrategy("Random Noise")

    def register_callbacks(self):
        def update_evasion_strategy(change):
            if change["new"] == "Random Noise":
                session.chooseStrategy("Random Noise")
            elif change["new"] == "General Search":
                session.chooseStrategy("search")
                session.chooseTransformation("genAlg")
                config = session.transformation.ga_config()
                config["numtuples"] = 4
                config["noise"] = 1
                config["features"] = [1, 1, 1, 1]
                config["predictor"] = session.prediction_proba
                config["oriVariant"] = 1
                config["weights"] = (-1.0, 1.0, 1.0)
                config["numberVariants"] = 1
                config["shift"] = 0
                config["numgen"] = 50
                config["population_size"] = 100
                config["lambda_sel"] = 50
                config["mu_sel"] = 50
                config["class"] = 1
                config["mutation_rate"] = 0.1

                session.setupTransformation(config)

                self.configuration["noise"] = 1.0
                self.configuration["shift"] = 0.0

                self.evasion_container.children = [
                    self.evasion_strategy_select,
                    self.evasion_select,
                ]
            items = list(map(lambda x: {"Feature": x}, session.data.getColumns()))
            self.evasion_table.items = items
            self.evasion_table.headers = [{"text": "Feature", "value": "Feature"}]

        def update_evasion(change):
            if change["new"] == "Discrete":
                session.chooseTransformation("discrete")
                self.configuration["noise"] = 1
                self.configuration["shift"] = 0

                self.evasion_container.children = [
                    self.evasion_strategy_select,
                    self.evasion_select,
                    paragraph("Select the number of variants per input:"),
                    self.number_variants_slider,
                    paragraph("Select the level of noise:"),
                    self.discrete_noise_slider,
                    paragraph("Select the shift:"),
                    self.discrete_shift_slider,
                ]
            elif change["new"] == "Continuous":
                session.chooseTransformation("continuous")
                self.configuration["noise"] = 1.0
                self.configuration["shift"] = 0.0

                self.evasion_container.children = [
                    self.evasion_strategy_select,
                    self.evasion_select,
                    paragraph("Select the number of variants per input:"),
                    self.number_variants_slider,
                    paragraph("Select the level of noise:"),
                    self.continuous_noise_slider,
                    paragraph("Select the shift:"),
                    self.continuous_shift_slider,
                ]

            items = list(map(lambda x: {"Feature": x}, session.data.getColumns()))
            self.evasion_table.items = items
            self.evasion_table.headers = [{"text": "Feature", "value": "Feature"}]

        def update_number_variants(*ignored):
            self.configuration["numberVariants"] = self.number_variants_slider.value

        def update_discrete_noise(*ignored):
            self.configuration["noise"] = self.discrete_noise_slider.value

        def update_continuous_noise(*ignored):
            self.configuration["noise"] = self.continuous_noise_slider.value

        def update_discrete_shift(*ignored):
            self.configuration["shift"] = self.discrete_shift_slider.value

        def update_continuous_shift(*ignored):
            self.configuration["shift"] = self.continuous_shift_slider.value

        def select_features(*ignored):
            columns = session.data.getColumns()
            features = np.repeat(0, len(columns))

            for element in self.evasion_table.v_model:
                features[columns.tolist().index(element["Feature"])] = 1

            self.configuration["features"] = features

        self.evasion_strategy_select.observe(update_evasion_strategy, "v_model")
        self.evasion_select.observe(update_evasion, "v_model")
        self.number_variants_slider.observe(update_number_variants, "value")
        self.discrete_noise_slider.observe(update_discrete_noise, "value")
        self.continuous_noise_slider.observe(update_continuous_noise, "value")
        self.discrete_shift_slider.observe(update_discrete_shift, "value")
        self.continuous_shift_slider.observe(update_continuous_shift, "value")
        self.select_features_button.on_event("click", select_features)

        # RUN section
        def run_code(*ignored):
            new_case = {}

            session.setupTransformation(self.configuration)
            session.data.transform(session.transformation)

            predictions = session.prediction(np.asarray(session.data.getVariants()))
            variants = session.data.getVariants()

            target_original = session.data.target

            variants_data_frame = pd.DataFrame(variants)
            columns = session.data.getColumns()
            
            variants_data_frame = variants_data_frame.set_axis(session.data.getColumns().values.tolist(), axis='columns')
            
            variants_data_frame["Original"] = target_original
            variants_data_frame["Predicted"] = session.prediction(session.data.data)
            variants_data_frame["Manipulation"] = predictions

            new_case["EvaAccuracy"] = sum(
                variants_data_frame["Original"] == variants_data_frame["Manipulation"]) * 100 / len(
                variants_data_frame["Original"])

            new_case["OriAccuracy"] = sum(
                variants_data_frame["Original"] == variants_data_frame["Predicted"]) * 100 / len(
                variants_data_frame["Original"])

            variants_data_frame["Success"] = variants_data_frame["Predicted"] != variants_data_frame["Manipulation"]

            headers = [
                {'text': 'Original', 'value': 'Original'},
                {'text': 'Predicted', 'value': 'Predicted'},
                {'text': 'Manipulation', 'value': 'Manipulation'},
                {'text': 'Success', 'value': 'Success'}
            ]
            items = list(map(lambda x: {'text': x, 'value': x}, session.data.getColumns()))

            self.variants_table.headers = items + headers
            self.variants_table.items = variants_data_frame.to_dict('records')

            variants_classes = np.unique(variants_data_frame["Manipulation"])
            prediction_classes = np.unique(variants_data_frame["Predicted"])

            matrix_heap_original = matrix_heap_variants = pd.DataFrame(
                np.zeros(
                    (len(prediction_classes), len(variants_classes))
                ), columns=variants_classes
            )

            for i in range(0, len(variants_data_frame["Manipulation"])):
                if variants_data_frame["Manipulation"][i] != variants_data_frame["Predicted"][i]:
                    matrix_heap_variants[variants_data_frame["Predicted"][i]][
                        variants_data_frame["Manipulation"][i]] += 1

            for i in range(0, len(variants_data_frame["Manipulation"])):
                if variants_data_frame["Original"][i] != variants_data_frame["Predicted"][i]:
                    matrix_heap_original[variants_data_frame["Predicted"][i]][variants_data_frame["Original"][i]] += 1

            new_case["EvaHeap"] = matrix_heap_variants
            new_case["OriHeap"] = matrix_heap_original
            new_case["EvasionStrategy"] = self.evasion_strategy_select.v_model
            new_case["EvasionType"] = self.evasion_select.v_model
            new_case["Noise"] = self.configuration["noise"]
            new_case["Shift"] = self.configuration["shift"]

            ModelTestingArea.evasion_cases["Evasion" +
                                           str(len(ModelTestingArea.evasion_cases) + 1)] = new_case

            ModelTestingArea.evaluation_report_select.items = list(ModelTestingArea.evasion_cases.keys())

        self.run_button.on_event("click", run_code)

    def content(self):
        return tabs(tab_headers=['Evasion', 'Run'], tab_items=[
            v.Layout(
                row=True,
                wrap=True,
                align_center=True,
                justify_center=True,
                class_='mx-2 my-2',
                children=[
                    v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                        v.Row(cols=2, lg=2, md=1, class_="my-8 justify-center", children=[
                            v.Col(class_="my-8 justify-center", children=[self.evasion_container]),
                            v.Col(class_="my-8 justify-center", children=[
                                centre(self.select_features_button, class_="mb-8"),
                                self.evasion_table,
                            ]),
                        ]),
                    ]),
                ]
            ),
            v.Layout(
                row=True,
                wrap=True,
                align_center=True,
                justify_center=True,
                class_='mx-2 my-2',
                children=[
                    v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                        v.Col(class_="my-8 justify-center", children=[
                            centre(self.run_button),
                            self.variants_table,
                        ]),
                    ]),
                ]
            ),
        ])

In [24]:
evasion = Evasion()

### Report

In [25]:
class Report(Page):
    help_content = "This part shows the results in terms of robustness and accuracy for your adversarial attack. The higher the robustness the better the model is to resists the attack. If you want to see any of the successful variants, go to the variants tab."

    def help(self):
        return v.Layout(
            row=True,
            wrap=True,
            align_center=True,
            justify_center=True,
            children=[
                v.Flex(xs12=True, lg6=True, xl4=True, pa_4=True, children=[
                    paragraph(self.help_content),
                ]),
            ],
        )

    def title(self):
        return "Model Blind Spots"

    def subtitle(self):
        return "Report"

    def name(self):
        return "Report"

    def icon(self):
        return "report"

    def __init__(self):
        super().__init__()

        font_style = {"font-size": "24px", "axes-size": "24px", "fill": "white", "font-weight": "bold"}

        self.original_heat_map = plt.figure(padding_y=0.0)
        self.original_heat_map.grid_map = plt.gridheatmap(
            pd.DataFrame(np.zeros((3, 3)), columns=["a", "b", "c"]),
            opacity=0.95,
            stroke="white",
        )
        self.original_heat_map.grid_map.display_format = ".2f"
        self.original_heat_map.grid_map.font_style = font_style

        self.variant_heat_map = plt.figure(padding_y=0.0)
        self.variant_heat_map.grid_map = plt.gridheatmap(
            pd.DataFrame(np.zeros((3, 3)), columns=["a", "b", "c"]),
            opacity=0.95,
            stroke="white",
        )
        self.variant_heat_map.grid_map.display_format = ".2f"
        self.variant_heat_map.grid_map.font_style = font_style

        self.evasion_report = v.Container(children=[])

        self.register_callbacks()

    def register_callbacks(self):
        def update_report(change):
            self.set_report(ModelTestingArea.evasion_cases[change["new"]])

        ModelTestingArea.evaluation_report_select.observe(update_report, "v_model")

    def content(self):
        return tabs(tab_headers=["Selection", "Report"], tab_items=[
            v.Layout(row=True, wrap=True, align_center=True, children=[
                v.Flex(xs12=True, lg6=True, xl4=True, class_="mx-4 my-4", children=[
                    v.Container(children=[
                        centre(ModelTestingArea.evaluation_report_select)
                    ])
                ]),
            ]),
            v.Layout(row=True, wrap=True, align_center=True, children=[
                v.Flex(xs12=True, lg6=True, xl4=True, class_="mx-4 my-4", children=[
                    self.evasion_report
                ]),
            ]),
        ])

    def draw_heat_map(self, evasion_case):
        self.original_heat_map.grid_map.color = evasion_case["OriHeap"]
        self.variant_heat_map.grid_map.color = evasion_case["EvaHeap"]

    def set_report(self, evasion_case):
        title = heading("Evasion Report", 1)
        strategy = heading(f"Strategy: {evasion_case['EvasionStrategy']}", 3)
        evasion_type = heading(f"Evasion: {evasion_case['EvasionType']}", 3)
        noise = heading(f"Noise: {evasion_case['Noise']}", 3)
        shift = heading(f"Shift: {evasion_case['Shift']}", 3)
        original_accuracy = heading(f"Original Accuracy: {evasion_case['OriAccuracy']}", 3)
        evasion_accuracy = heading(f"Variants Accuracy: {evasion_case['EvaAccuracy']}", 3)

        self.draw_heat_map(evasion_case)

        self.evasion_report.children = [
            title,
            vspace(),
            strategy,
            evasion_type,
            noise,
            shift,
            original_accuracy,
            evasion_accuracy,
            heading("Original Misclassification HeatMap: ", 3),
            self.original_heat_map,
            heading("Variants Misclassification HeatMap: ", 3),
            self.variant_heat_map,
        ]

In [26]:
report = Report()

### Add Model Testing pages to site

In [27]:
model_testing_links = [
    home,
    dataset,
    model,
    evasion,
    report
]

dataset.add_navigation_pages(*model_testing_links)
model.add_navigation_pages(*model_testing_links)
evasion.add_navigation_pages(*model_testing_links)
report.add_navigation_pages(*model_testing_links)

home.add_navigation_pages(dataset)

## Render site

In [28]:
site.content()

Layout(children=[Layout(align_center=True, children=[Flex(children=[Html(children=[Img(alt='MLighter logo', la…

## Run Unit Tests

Optionally run unit tests with ipytest  

In [29]:
# button to optionally run all tests
run_all_button = v.Btn(
    color="primary",
    children=[
        "Run tests"
    ]
)

def run_all_tests(*ignored):
    run_tests()
    
run_all_button.on_event("click", run_all_tests)

run_all_button

Btn(children=['Run tests'], color='primary', layout=None)